In [1]:
import torch
import pytorch_lightning as pl
import torchvision
from torchvision.datasets import MNIST
from matplotlib import pyplot as plt
from pytorch_lightning.loggers import TensorBoardLogger

import numpy as np
from torch.nn import functional as F
from torch import nn
import foolbox as fb
import torchvision.models as models

In [2]:
class MNISTClassifer(pl.LightningModule):
    def __init__(self, regular_train=True, adv_train=False):
        super(MNISTClassifer, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=5, padding=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features=28*28*64, out_features=128)
        self.out = nn.Linear(in_features=128, out_features=10)
        self.attack_type = fb.attacks.L2FastGradientAttack()
        self.adv_train = adv_train
        self.regular_train = regular_train

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.dropout(x, 0.25)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.out(x)
        return F.log_softmax(x, dim=1)


    def prepare_data(self):
        self.train_data = MNIST(root="data/", train=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))

        self.val_data = MNIST(root="data/", train=False, transform=torchvision.transforms.Compose([ torchvision.transforms.ToTensor()]))

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_data, batch_size=128, shuffle=True, num_workers=12)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_data, batch_size=128, shuffle=False, num_workers=12)


    def training_step(self, train_batch, batch_idx):
        
        return_dict = {}

        data, label = train_batch

        if self.regular_train:
            out = self.forward(data)
            loss = F.nll_loss(out, label)
            prediction = out.argmax(dim=1, keepdim=True).squeeze()
            correct =  prediction.eq(label.view_as(prediction)).sum().item()
            return_dict['loss'] = loss 
            return_dict['correct'] = correct 

        if self.adv_train:
            self.eval()
            adv_model = fb.PyTorchModel(self, bounds=(0,1))
            adv_data, _, success = self.attack_type(adv_model, data, label, epsilons=[0.01])
            print(adv_data)
            successful_attack_no = torch.sum(success.long())
            self.zero_grad()
            out = self.forward(adv_data)
            adv_loss = F.nll_loss(out, label)
            return_dict['successful_attack'] = successful_attack_no
            return_dict['adv_loss'] = adv_loss

        return return_dict

    
    def training_epoch_end(self, outputs):
        #end of an epoch
        #calculate the average loss of this epoch}
        return_dict = {}
        logs = {}
        if self.regular_train:
            avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
            accuracy = 100 * (sum([x['correct'] for x in outputs]) / float(len(self.train_data)))
            logs['train_loss'] = avg_loss
            logs['train_accuracy'] = accuracy
            return_dict['loss'] = avg_loss
            
        if self.adv_train:
            adv_avg_loss = torch.stack([x['adv_loss'] for x in outputs]).mean()
            adv_accuracy = (sum([x['successful_attack'] for x in outputs]) / float(len(self.train_data)))
            logs['adv_loss'] = adv_avg_loss
            logs['robust_accuracy'] = 100. * (1. - adv_accuracy)
            return_dict['adv_loss'] = adv_loss

                
        return {'loss' : avg_loss, 'log' : logs}


    def validation_step(self, validation_batch, batch_idx):
        if batch_idx == 0:
            torch.cuda.memory_summary()
        
        data, label = validation_batch
        #standard inference
        out = self.forward(data)
        loss = F.nll_loss(out, label)
        prediction = out.argmax(dim=1, keepdim=True).squeeze()
        correct =  prediction.eq(label.view_as(prediction)).sum().item()

        return {'loss' : loss, 'correct' : correct}



    def validation_epoch_end(self, outputs):
        #end of an epoch
        #calculate the average loss of this epoch
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        accuracy = 100. * (sum([x['correct'] for x in outputs]) / float(len(self.val_data)))

        logs = {'validation_loss': avg_loss, 'validation_accuracy': accuracy}
        return {'loss' : avg_loss, 'log' : logs}

    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())


    def adversarial_validation(self):
        val_dataloader = self.val_dataloader()
        self.eval()
        adv_model = fb.PyTorchModel(self, bounds=(0,1))
        successful_attack_sum = 0
        
        with torch.enable_grad():
            for batch_id, (data, label) in enumerate(val_dataloader):
                data, label = data.cuda(), label.cuda()
                _, _, success = self.attack_type(adv_model, data, label, epsilons=[0.01])
                successful_attack_no = torch.sum(success.long())

                successful_attack_sum += successful_attack_no

        self.zero_grad()

        print("Successful attack:{} Attack count:{} Percentage:{}".format(successful_attack_sum, len(self.val_data), float(successful_attack_sum) /len(self.val_data)))
        robust_accuracy = 100. * (1-(float(successful_attack_sum) /len(self.val_data))) 
        return robust_accuracy





In [3]:
model = MNISTClassifer(regular_train=True, adv_train=True)

In [4]:
model.prepare_data()

In [5]:
logger = TensorBoardLogger('robust_gan_logs', name="mnist_classifier")

In [8]:
trainer = pl.Trainer(max_epochs=6, logger=logger, gpus=[0], fast_dev_run=True)


Running in fast_dev_run mode: will run a full train, val and test loop using a single batch


MisconfigurationException: 
                You requested GPUs: [0]
                But your machine only has: []
            

In [7]:
trainer.fit(model)

NameError: name 'trainer' is not defined

In [14]:
model.adversarial_validation()

Successful attack:103 Attack count:10000 Percentage:0.0103


98.97